## project

In [18]:
%env DD_TOKEN=https://oapi.dingtalk.com/robot/send?access_token=1003a3d8a6e724e255d9b28b5d4a540cd8dbdd2d2fa2163ed09cc77f7c46841d

env: DD_TOKEN=https://oapi.dingtalk.com/robot/send?access_token=1003a3d8a6e724e255d9b28b5d4a540cd8dbdd2d2fa2163ed09cc77f7c46841d


In [19]:
from zxftools.llms import get_llm
import os
import subprocess

In [20]:
import re
def get_code(result):
    if verbose:
        print(result)
    code_match = re.search(r'### 3. Function\n\n```python(.*?)```',result,flags = re.DOTALL)
    return code_match.group(1)

<font color="orange"> update1 添加可用的函数 </font>

<font color="orange"> update2  </font>

In [21]:
import random

In [ ]:
## external_functions

有一个提取模块,用来从大量的函数描述中提取可用的函数

In [22]:
external_functions = """

    """
suggest = """
判断输入的信息 是否可以通过数据库查到

这是数据库的查询方法

def get_sql_app(app_name):#暂时app的id没有暴露

    url = "http://180.184.80.35:6688/aiquant-mark/page-data/queryApp"  
    param={'appName':app_name}
    response = requests.get(url,params=param)  
    return response.json()
    
返回格式:
{'code': '00000000',
 'message': '成功',
 'requestId': '0f805c8994924193a0b9d7b86d61eb83',
 'data': [{'id': 1,
   'appName': 'Ahrefs',
   'appDesc': 'Ahrefs API: SEO powerhouse featuring keyword research, backlink analysis, and SERP tracking tools for maximizing website visibility and rankings.',
   'appIcon': 'https://rapidapi-prod-apis.s3.amazonaws.com/de36eede-a1d6-4818-bd80-d5a9edddcf0a.png',
   'appNum': 11,
   'appUrl': 'https://rapidapi.com/Glavier/api/ahrefs1',
   'createTime': '2024-07-10 19:48:45',
   'modifyTime': ''}]}
    """

In [23]:
def write_code(main_path,llm):
    with open(os.path.join(main_path,'test.py'),'r') as f:
        unittest_code = f.read()
        
    result = llm.complete(f"""
You are a python programer helping user to deal problem by create a python function.
you can use existing utility functions when you need.
{external_functions}

You can refer to these ideas and suggestions.
{suggest}

Following '===' is the unittest code. 
Only use the unittest code between first and second '===' to accomplish the task above, do not Mixed into the python function.


===
{unittest_code}
===

Never forget that your functions should be robust and adaptable. your goal is to pass the unit tests.

Your results should be kept in four parts: 
    1 your deconstruction and understanding of the problem
    2 your function design ideas (pseudocode)
    3 your formal function
    4 your function call examples
    
Example:

### 1. Understand
The task is xxx
### 2. Design
1. xxxx
2. xxxx
3. xxxx
### 3. Function

```python
import re

def prepend_to_file(file_path, data_to_prepend,new_file_path=None):
    
    if new_file_path is None:
        new_file_path = file_path
    # 读取原始文件内容
    with open(file_path, 'r', encoding='utf-8') as file:
        original_content = file.read()
    
    # 将新数据添加到原始内容前面
    updated_content = data_to_prepend +'\n'+ original_content
    
    # 将更新后的内容写回文件
    with open(new_file_path, 'w', encoding='utf-8') as file:
        file.write(updated_content)
```
### 4. Function Call Examples
```python
xxx
```
    
""").text
    return get_code(result)

In [24]:
def rewrite_code(code,error_info,llm):
    result = llm.complete(f"""
You are a python programer helping user to deal problem by check python functions ,eliminate bugs and fix errors.
Following '===' is the user's code and execution error result. 
Only use the user's code and execution error result between first and second '===' to accomplish the task above, do not Mixed into the python function.

===
{code}


{error_info}
===



Never forget that your functions should be robust and adaptable. your goal is to pass the unit tests.

Your results should be kept in four parts: 
    1 your deconstruction and understanding of the problem
    2 your function design ideas (pseudocode)
    3 your formal function
    4 your function call examples
    
Example:

### 1. Understand
The task is xxx
### 2. Design
1. xxxx
2. xxxx
3. xxxx
### 3. Function

```python
import re

def prepend_to_file(file_path, data_to_prepend,new_file_path=None):
    
    if new_file_path is None:
        new_file_path = file_path
    # 读取原始文件内容
    with open(file_path, 'r', encoding='utf-8') as file:
        original_content = file.read()
    
    # 将新数据添加到原始内容前面
    updated_content = data_to_prepend +'\n'+ original_content
    
    # 将更新后的内容写回文件
    with open(new_file_path, 'w', encoding='utf-8') as file:
        file.write(updated_content)
```
### 4. Function Call Examples
```python
xxx
```
    
""").text
    return get_code(result)

In [25]:
def prepend_to_file(file_path, data_to_prepend,new_file_path=None):
    
    if new_file_path is None:
        new_file_path = file_path
    # 读取原始文件内容
    with open(file_path, 'r', encoding='utf-8') as file:
        original_content = file.read()
    
    # 将新数据添加到原始内容前面
    updated_content = data_to_prepend +'\n'+ original_content
    
    # 将更新后的内容写回文件
    with open(new_file_path, 'w', encoding='utf-8') as file:
        file.write(updated_content)
        
    cc = """
if __name__ == '__main__':
    unittest.main()
"""
    with open(new_file_path,'a', encoding='utf-8') as file:
        file.write(cc)


### RUN

In [26]:
llm = get_llm(model='gpt-4o',temperature=0.01)

func_name = 'wake_function_5' # TODO 函数名
verbose = True # TODO

In [27]:
from zxftools.project import DDMessage

robot = DDMessage()
main_path = os.path.join('./function/',func_name)

if not os.path.exists(main_path):
    os.makedirs(main_path)

In [28]:
%%writefile {main_path}/test.py  
import unittest

class TestFunction(unittest.TestCase):
    
    def test(self):
        inputs = 'Ahrefs'
        result = func(inputs)
        self.assertTrue(result)

    def test2(self):
        inputs = 'Ahrxv'
        result = func(inputs)
        self.assertFalse(result)

    

if __name__ == '__main__':
    unittest.main()


Writing ./function/wake_function_5/test.py


In [29]:
# 函数名应该叫func

In [ ]:
therm thermo heat 热的

thermal 热的

In [30]:
for i in range(3):
    if i == 0:
        python_code = write_code(main_path,llm)
        prepend_to_file(os.path.join(main_path,'test.py'), 
                        python_code,
                        os.path.join(main_path,f'test_run_{i}.py'))
        unittest_result = subprocess.run(f"python {main_path}/test_run_{i}.py", shell=True, text=True, capture_output=True)
    # 先不考虑修改
#     else:
#         with open(f'{main_path}/test_run_{i-1}.py','r') as f:
#             code = f.read()
#         print(f'第{i}轮修改')
#         print(unittest_result.stderr)

#         python_code = rewrite_code(code,unittest_result.stderr,llm)
#         prepend_to_file(os.path.join(main_path,'test.py'), 
#                         python_code,
#                         os.path.join(main_path,f'test_run_{i}.py'))
#         unittest_result = subprocess.run(f"python {main_path}/test_run_{i}.py", shell=True, text=True, capture_output=True)
    else:
        print('error')
    if unittest_result.returncode == 0:
        func_name = main_path.split('/')[-1]
        with open(os.path.join(main_path,f"completed_function.py"),'w') as f:
            f.write(python_code.replace('func',func_name))
        robot.send(f'函数{func_name} 的编写以完成,已保存至{os.path.join(main_path,f"completed_function.py")}中')
        break  # 如果条件为假，跳出循环

### 1. Understand
The task is to create a Python function that checks if a given app name can be found in a database using a provided API endpoint. The function should return `True` if the app is found and `False` otherwise. The function should be robust enough to handle different inputs and pass the provided unit tests.

### 2. Design
1. Define the function `func` that takes an app name as input.
2. Use the provided `get_sql_app` function to query the database with the given app name.
3. Check the response from the `get_sql_app` function:
   - If the response code is '00000000' and the data list is not empty, return `True`.
   - Otherwise, return `False`.

### 3. Function

```python
import requests

def get_sql_app(app_name):
    url = "http://180.184.80.35:6688/aiquant-mark/page-data/queryApp"
    param = {'appName': app_name}
    response = requests.get(url, params=param)
    return response.json()

def func(app_name):
    response = get_sql_app(app_name)
    if response['code'] == 

In [31]:
print(python_code)


import requests

def get_sql_app(app_name):
    url = "http://180.184.80.35:6688/aiquant-mark/page-data/queryApp"
    param = {'appName': app_name}
    response = requests.get(url, params=param)
    return response.json()

def func(app_name):
    response = get_sql_app(app_name)
    if response['code'] == '00000000' and response['data']:
        return True
    return False



%load {main_path}/completed_function.py

<font color="yellow"> update1 完成  同时完成了传递函数功能的能力</font>

<font color="orange"> 优点在于精确 基本上对于小函数来说,将函数的编写转化为测试用例的编写</font>